In [1]:
from scipy.misc import imread, imsave
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import scipy.spatial
import sobol

##1. Parametric Kernel

In [24]:
# Matern kernel parameters
SIGMA_RHO = 4
SIGMA_TAU = 1
SIGMA_GAMMA = 1

class Theta:
    def __init__(self, gamma, rho, tau):
        self.gamma = gamma
        self.rho = rho
        self.tau = tau

# GP1
def r_squared(x, z, rho):
    return np.sum((x - z)**2) / (rho ** 2)
def G(x, z, gamma, rho):
    return np.exp(-r_squared(x, z, rho) ** gamma)
    
# GP2
def euclidean_distance(theta, theta_prime):
    sigma_rho = SIGMA_RHO
    sigma_tau = SIGMA_TAU
    sigma_gamma = SIGMA_GAMMA
    return (theta.rho - theta_prime.rho / sigma_rho) ** 2 + \
           (theta.gamma - theta_prime.gamma / sigma_gamma) ** 2 + \
           (theta.tau - theta_prime.tau / sigma_tau) ** 2

def matern_kernel(theta, theta_prime):
    s2 = euclidean_distance(theta, theta_prime)
    return (1 + np.sqrt(5*s2) + 5 / 3. * s2) * np.exp(-np.sqrt(5*s2))

In [31]:
def GP1_kernel(data, theta, max_distance):
    assert len(data.shape) == 2
    
    factor = -0.5 /  (theta.rho ** 2)
    limit = np.exp(factor*max_distance**2)
    # Find the pairwise squared distances and compute the Gaussian kernel.
    K = []
    for x in data:
        d = np.array([G(x, z, theta.gamma, theta.rho) for z in data])
        d[d < limit] = 0.0  # truncate the Gaussian
        d = scipy.sparse.csc_matrix(d[:,None])
        K.append(d)
    K = scipy.sparse.hstack(K)
    return K
    
def compute_alpha(train_x, train_y, theta, max_distance):
    print "building input kernel matrix"
    K = GP1_kernel(train_x, theta, max_distance)
    print "sparsity: %.2f%%" % (float(100*K.nnz) / (K.shape[0]*K.shape[1]))
    M = K + theta.tau * scipy.sparse.identity(train_x.shape[0])
    y = scipy.sparse.csc_matrix(train_y[:,None])
    print "solving sparse system"
    alpha = scipy.sparse.linalg.cg(M, train_y)
    print "done computing alpha"
    return alpha[0]

class ExponentialKernelRegressor(object):
    def __init__(self, theta):
        self.dim = None
        self.train_x = None
        self.alpha = None
        self.mean_y = None
        self.std_y = None
        self.theta = theta
        self.max_distance = 4.0 * self.theta.rho

    def train(self, train_x, train_y):
        """Train the exponential kernel regressor.

        :param train_x: training x data
        :param train_y: training y data
        """
        assert len(train_x.shape) == 2
        assert len(train_y.shape) == 1
        assert train_x.shape[0] == train_y.shape[0]
        
        self.dim= train_x.shape[1]
        self.train_x = train_x.astype(np.float32)
        self.tree = scipy.spatial.cKDTree(self.train_x)
        
        self.mean_y = train_y.mean()
        self.std_y = train_y.std()
        train_y_std = (train_y - self.mean_y) / self.std_y
        
        self.alpha = compute_alpha(self.train_x, train_y_std, self.theta, self.max_distance)
        
    def predict_single(self, pred_x):
        """Predict the value of a single instance.

        :param pred_x: x data
        :return: predicted value of pred_x
        """
        assert len(pred_x.shape) == 1
        assert pred_x.shape[0] == self.dim
        indices = np.asarray(self.tree.query_ball_point(pred_x, self.max_distance))
        dist = np.sum((self.train_x[indices]-pred_x)**2, axis=1)
        kappa = np.exp(self.scale*dist)
        pred_y = np.dot(kappa, self.alpha[indices])
        return self.std_y * pred_y + self.mean_y

    def predict(self, pred_x):
        """Predict the values of pred_x.

        :param pred_x: x data
        :return: predicted values of pred_x
        """
        assert len(pred_x.shape) == 2
        assert pred_x.shape[1] == self.dim
        pred_x = pred_x.astype(np.float32)
        return np.array([self.predict_single(x) for x in pred_x])

def exponetial_kernel_regression(tau, rho, gamma, verbose=False):
    # Load the image.
    im_orig = np.squeeze(imread("cc_90.png"))

    # Make a copy, so both the original and the regressed image can be shown afterwards.
    im = np.array(im_orig)

    # Find the known pixels and the pixels that shall be predicted.
    known_ind = np.where(im != 0)
    unknown_ind = np.where(im >= 0)
    known_x = np.array(known_ind).transpose()
    known_y = np.array(im[known_ind])
    pred_x = np.array(unknown_ind).transpose()
    
    theta = Theta(gamma, rho, tau)
    reg = ExponentialKernelRegressor(theta)
    
    reg.train(known_x, known_y)
    
    pred_y = reg.predict(pred_x)
    
    im[unknown_ind] = pred_y
    
    if verbose:
        imsave("res.png", im)
    
    return im

In [32]:
test = exponetial_kernel_regression(0.8, 7.5, 1.8)

building input kernel matrix


KeyboardInterrupt: 

## Bayesian Optimization of Hyperparameters

In [7]:
def mse(I, I_head):
    assert(I.shape == I_head.shape)
    
    return [(I[i, j] - I_head[i, j]) ** 2 for i in range(I.shape[0] for j in range(I.shape[1]))] / float(I.size)

In [ ]:
NUM_SAMPLES = 2000
NUM_P = 20
LAMBDA = 0.3

parameterUpperLimits = np.array([limit_rho, limit_gamma, limit_tau])
parameterLowerLimits = np.array([limit_rho, limit_gamma, limit_tau])
for i in range ( number_of_samples ):
    rho , gamma , tau = sobol.i4_sobol(3,i)[0] * (parameterUpperLimits - parameterLowerLimits) + parameterLowerLimits

msea = E.T * np.invert(M + LAMBDA * np.identity(thetas.size)) * m